In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [35]:
url = "https://raw.githubusercontent.com/HarshTiwari1710/Sobershot/refs/heads/main/mr-boston-flattened.csv"

In [36]:
drinks_df = pd.read_csv(url)

In [37]:
drinks_df

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,Juice of a Lime,1/2 oz,Powdered Sugar,2 oz,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Wallis Blue Cocktail,Gin,1,"Lime wedge, superfine sugar",1 oz,Gin,1 oz,Triple Sec,1 oz,Fresh Lime Juice,NaN,NaN,NaN,NaN,Rim old-fashioned glass with lime and sugar. F...,Old-Fashioned Glass,6 to 8 ounces
986,Minnehaha Cocktail,Cocktail Classics,1/4 oz,Juice of Orange,1/2 oz,Dry Vermouth,1/2 oz,Sweet Vermouth,1 oz,Old Mr. Boston Dry Gin,NaN,NaN,NaN,NaN,Shake well with cracked ice and strain into 4 ...,Cocktail Glass,6 or more ounces
987,Wallick Cocktail,Gin,1 1/2 oz,Gin,1 1/2 oz,Dry Vermouth,1 oz,Triple Sec,NaN,NaN,NaN,NaN,NaN,NaN,Stir with ice and strain into chilled cocktail...,Cocktail Glass,6 or more ounces
988,Waikiki Beachcomber,Gin,3/4 oz,Gin,3/4 oz,Triple Sec,1/2 oz,Pineapple Juice,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into chilled cocktai...,Cocktail Glass,6 or more ounces


In [38]:
drinks_df.isnull().sum()

name               0
category           0
measurement-1     10
ingredient-1       1
measurement-2     35
ingredient-2       3
measurement-3    143
ingredient-3      96
measurement-4    390
ingredient-4     406
measurement-5    683
ingredient-5     669
measurement-6    919
ingredient-6     831
instructions       0
glass              5
glass-size        11
dtype: int64

In [39]:
drinks_df.describe()

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
count,990,990,980,989,955,987,847,894,600,584,307,321,71,159,990,985,979
unique,990,11,33,237,19,256,22,227,47,173,44,144,12,117,580,20,16
top,Gauguin,Cocktail Classics,1 1/2 oz,Vodka,1 oz,Dry Vermouth,1/2 oz,Fresh lemon juice,1 oz,Fresh lemon juice,1 oz,Angostura Bitters,1 oz,Lemon twist,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
freq,1,468,289,95,291,63,267,57,195,33,101,24,32,7,115,542,542


In [40]:
drinks_df.columns

Index(['name', 'category', 'measurement-1', 'ingredient-1', 'measurement-2',
       'ingredient-2', 'measurement-3', 'ingredient-3', 'measurement-4',
       'ingredient-4', 'measurement-5', 'ingredient-5', 'measurement-6',
       'ingredient-6', 'instructions', 'glass', 'glass-size'],
      dtype='object')

In [41]:
drinks_df

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,Juice of a Lime,1/2 oz,Powdered Sugar,2 oz,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Wallis Blue Cocktail,Gin,1,"Lime wedge, superfine sugar",1 oz,Gin,1 oz,Triple Sec,1 oz,Fresh Lime Juice,NaN,NaN,NaN,NaN,Rim old-fashioned glass with lime and sugar. F...,Old-Fashioned Glass,6 to 8 ounces
986,Minnehaha Cocktail,Cocktail Classics,1/4 oz,Juice of Orange,1/2 oz,Dry Vermouth,1/2 oz,Sweet Vermouth,1 oz,Old Mr. Boston Dry Gin,NaN,NaN,NaN,NaN,Shake well with cracked ice and strain into 4 ...,Cocktail Glass,6 or more ounces
987,Wallick Cocktail,Gin,1 1/2 oz,Gin,1 1/2 oz,Dry Vermouth,1 oz,Triple Sec,NaN,NaN,NaN,NaN,NaN,NaN,Stir with ice and strain into chilled cocktail...,Cocktail Glass,6 or more ounces
988,Waikiki Beachcomber,Gin,3/4 oz,Gin,3/4 oz,Triple Sec,1/2 oz,Pineapple Juice,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into chilled cocktai...,Cocktail Glass,6 or more ounces


In [42]:
def combine_ingredients_to_dict(row, num_pairs=6):
    ingredients_dict = {}
    for i in range(1, num_pairs + 1):
        ingredient_col = f'ingredient-{i}'
        measurement_col = f'measurement-{i}'
        if ingredient_col in row:
            ingredient = row[ingredient_col]
            measurement = row.get(measurement_col, "1 oz")  # Default to "1 oz" if measurement is missing
            if pd.notna(ingredient):
                if pd.isna(measurement):
                    measurement = "1 oz"  # Assign "1 oz" if the measurement is NaN
                ingredients_dict[ingredient] = measurement
    return ingredients_dict

# Assuming 'drinks_df' is your DataFrame loaded with data
# Apply the function to each row to create a new column for the ingredient-measurement dictionary
drinks_df['ingredient_dict'] = drinks_df.apply(combine_ingredients_to_dict, axis=1)

# Drop the original ingredient and measurement columns
for i in range(1, 7):
    drinks_df.drop([f'ingredient-{i}', f'measurement-{i}'], axis=1, inplace=True)



In [43]:
drinks_df

,name,category,instructions,glass,glass-size,ingredient_dict
0,Gauguin,Cocktail Classics,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces,"{' Light Rum': '2 oz', ' Passion Fruit Syrup':..."
1,Fort Lauderdale,Cocktail Classics,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces,"{' Light Rum': '1 1/2 oz', ' Sweet Vermouth': ..."
2,Apple Pie,Cordials and Liqueurs,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces,"{' Apple schnapps': '3 oz', ' Cinnamon schnapp..."
3,Cuban Cocktail No. 1,Cocktail Classics,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces,"{' Juice of a Lime': '1/2 oz', ' Powdered Suga..."
4,Cool Carlos,Cocktail Classics,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces,"{' Dark rum': '1 1/2 oz', ' Cranberry Juice': ..."
...,...,...,...,...,...,...
985,Wallis Blue Cocktail,Gin,Rim old-fashioned glass with lime and sugar. F...,Old-Fashioned Glass,6 to 8 ounces,"{'Lime wedge, superfine sugar': '1', ' Gin': '..."
986,Minnehaha Cocktail,Cocktail Classics,Shake well with cracked ice and strain into 4 ...,Cocktail Glass,6 or more ounces,"{' Juice of Orange': '1/4 oz', ' Dry Vermouth'..."
987,Wallick Cocktail,Gin,Stir with ice and strain into chilled cocktail...,Cocktail Glass,6 or more ounces,"{' Gin': '1 1/2 oz', ' Dry Vermouth': '1 1/2 o..."
988,Waikiki Beachcomber,Gin,Shake with ice and strain into chilled cocktai...,Cocktail Glass,6 or more ounces,"{' Gin': '3/4 oz', ' Triple Sec': '3/4 oz', ' ..."


In [44]:
drinks_df.isnull().sum()

name                0
category            0
instructions        0
glass               5
glass-size         11
ingredient_dict     0
dtype: int64

In [45]:
mode_glass_size = drinks_df['glass-size'].mode()[0]  # The mode() method returns a Series, take the first element

# Fill missing values in 'glass-size' with the mode
drinks_df['glass-size'].fillna(mode_glass_size, inplace=True)

In [46]:
missing_glass = drinks_df[drinks_df['glass'].isnull()]

In [47]:
missing_glass

,name,category,instructions,glass,glass-size,ingredient_dict
53,Martini (Medium),Cocktail Classics,Follow directions for Martini preparation.,NaN,6 or more ounces,"{' Gin': '1 1/2 oz', ' Dry Vermouth': '1/2 oz'..."
54,Martini (Dry) (5-to-1),Cocktail Classics,Follow directions for Martini preparation.,NaN,6 or more ounces,"{' Gin': '1 2/3 oz', ' Dry Vermouth': '1/3 oz'}"
255,Dry Martini,Cocktail Classics,serve with an olive.,NaN,6 or more ounces,"{' Mr. Boston Gin': '1 2/3 oz', ' Dry Vermouth..."
258,Dillatini Cocktail,Cocktail Classics,Serve with a dilly bean.,NaN,6 or more ounces,"{' Mr. Boston Gin': '1 1/2 oz', ' Dry Vermouth..."
437,Vodka Martini,Cocktail Classics,serve with an olive.,NaN,6 or more ounces,"{' Mr. Boston Vodka': '1 1/2 oz', ' Dry Vermou..."


In [48]:
drinks_df['glass'].fillna('Martini Glass', inplace=True)

In [49]:
drinks_df.isnull().sum()

name               0
category           0
instructions       0
glass              0
glass-size         0
ingredient_dict    0
dtype: int64

In [50]:
print(drinks_df['ingredient_dict'])

0      {' Light Rum': '2 oz', ' Passion Fruit Syrup':...
1      {' Light Rum': '1 1/2 oz', ' Sweet Vermouth': ...
2      {' Apple schnapps': '3 oz', ' Cinnamon schnapp...
3      {' Juice of a Lime': '1/2 oz', ' Powdered Suga...
4      {' Dark rum': '1 1/2 oz', ' Cranberry Juice': ...
                             ...                        
985    {'Lime wedge, superfine sugar': '1', ' Gin': '...
986    {' Juice of Orange': '1/4 oz', ' Dry Vermouth'...
987    {' Gin': '1 1/2 oz', ' Dry Vermouth': '1 1/2 o...
988    {' Gin': '3/4 oz', ' Triple Sec': '3/4 oz', ' ...
989    {' Gin': '1 1/2 oz', ' Grapefruit Juice': '3/4...
Name: ingredient_dict, Length: 990, dtype: object


In [51]:
%pip install requests beautifulsoup4 fake-useragent
%pip install requests-html pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [52]:
import pandas as pd
import requests
import random
import time

def get_drink_image_url(drink_name: str, max_retries: int = 3) -> str:
    """
    Retrieve the first image URL for a drink using Serpapi (free alternative)
    
    Args:
        drink_name (str): Name of the drink to search
        max_retries (int): Maximum number of retry attempts
    
    Returns:
        str: Image URL or None if not found
    """
    # API key for SerpApi (free tier available)
    # Replace with your actual API key or use environment variable
    api_key = "1bbbb8e960727f679976af15eacefe07ac608f52"  
    
    # Search queries to increase chances of finding an image
    search_queries = [
        f"{drink_name} cocktail",
        f"{drink_name} drink",
        f"{drink_name} beverage"
    ]
    
    for attempt in range(max_retries):
        try:
            # Randomize search query
            query = random.choice(search_queries)
            
            # SerpApi parameters
            params = {
                "engine": "google_images",
                "q": query,
                "api_key": api_key
            }
            
            # Send request to SerpApi
            response = requests.get("https://google.serper.dev/search", params=params)
            response.raise_for_status()
            
            # Parse the response
            data = response.json()
            
            # Extract first image URL from results
            if 'images_results' in data and data['images_results']:
                return data['images_results'][0].get('original')
            
            # Add delay between attempts
            time.sleep(random.uniform(1, 3))
        
        except Exception as e:
            print(f"Error searching for {drink_name} (Attempt {attempt + 1}/{max_retries}): {e}")
            time.sleep(random.uniform(1, 3))
    
    return None

def add_drink_image_urls(drinks_df: pd.DataFrame, drink_column_name: str = 'name') -> pd.DataFrame:
    """
    Add image URLs to the DataFrame
    
    Args:
        drinks_df (pd.DataFrame): Input DataFrame
        drink_column_name (str): Column containing drink names
    
    Returns:
        pd.DataFrame: DataFrame with added image URLs
    """
    # Create a copy of the DataFrame
    df = drinks_df.copy()
    
    # Add image URLs column
    df['image_url'] = df[drink_column_name].apply(get_drink_image_url)
    
    return df

# Usage notes:
# 1. Sign up for a free SerpApi key at https://serpapi.com/
# 2. Replace 'YOUR_SERPAPI_KEY_HERE' with your actual API key
# 3. Install required library: pip install requests pandas
# 
# Example:


In [53]:
modified_drinks_df = drinks_df.copy()

# Modify the 'Drink Name' column to append "drink" to each name
modified_drinks_df['name'] = modified_drinks_df['name'] + " cocktail"

In [54]:
modified_drinks_df

,name,category,instructions,glass,glass-size,ingredient_dict
0,Gauguin cocktail,Cocktail Classics,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces,"{' Light Rum': '2 oz', ' Passion Fruit Syrup':..."
1,Fort Lauderdale cocktail,Cocktail Classics,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces,"{' Light Rum': '1 1/2 oz', ' Sweet Vermouth': ..."
2,Apple Pie cocktail,Cordials and Liqueurs,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces,"{' Apple schnapps': '3 oz', ' Cinnamon schnapp..."
3,Cuban Cocktail No. 1 cocktail,Cocktail Classics,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces,"{' Juice of a Lime': '1/2 oz', ' Powdered Suga..."
4,Cool Carlos cocktail,Cocktail Classics,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces,"{' Dark rum': '1 1/2 oz', ' Cranberry Juice': ..."
...,...,...,...,...,...,...
985,Wallis Blue Cocktail cocktail,Gin,Rim old-fashioned glass with lime and sugar. F...,Old-Fashioned Glass,6 to 8 ounces,"{'Lime wedge, superfine sugar': '1', ' Gin': '..."
986,Minnehaha Cocktail cocktail,Cocktail Classics,Shake well with cracked ice and strain into 4 ...,Cocktail Glass,6 or more ounces,"{' Juice of Orange': '1/4 oz', ' Dry Vermouth'..."
987,Wallick Cocktail cocktail,Gin,Stir with ice and strain into chilled cocktail...,Cocktail Glass,6 or more ounces,"{' Gin': '1 1/2 oz', ' Dry Vermouth': '1 1/2 o..."
988,Waikiki Beachcomber cocktail,Gin,Shake with ice and strain into chilled cocktai...,Cocktail Glass,6 or more ounces,"{' Gin': '3/4 oz', ' Triple Sec': '3/4 oz', ' ..."


In [62]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape images from
base_url = "https://www.thecocktaildb.com"

def scrape_image_urls(limit=1000):
    image_urls = []
    try:
        response = requests.get(f"{base_url}/drinks.php")
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all image tags
        images = soup.find_all('img')
        
        for img in images:
            if 'src' in img.attrs and len(image_urls) < limit:
                full_url = f"{base_url}{img['src']}"
                image_urls.append(full_url)
        
        # If the page doesn't have enough images, display a message
        if len(image_urls) < limit:
            print(f"Only {len(image_urls)} images were found on the page.")
        
    except Exception as e:
        print(f"Error during scraping: {e}")
    
    return image_urls

# Scrape up to 1000 image URLs
image_urls = scrape_image_urls(limit=1000)

# Create a dataframe with the scraped image URLs
drinks_df = pd.DataFrame({'Image URL': image_urls})

# Save the dataframe to a CSV file
file_path = 'scraped_drinks_images.csv'
drinks_df.to_csv(file_path, index=False)

print(f"Image URLs have been saved to {file_path}.")


Error during scraping: 404 Client Error: Not Found for url: https://www.thecocktaildb.com/drinks.php
Image URLs have been saved to scraped_drinks_images.csv.


In [61]:
image_df

,img_url


In [63]:
modified_drinks_df['img_url']

0      https://domesticgeekgirl.com/wp-content/upload...
1      https://drinksworld.com/wp-content/uploads/For...
2      https://thetoastykitchen.com/wp-content/upload...
3      http://www.thedrinkguy.com/wp-content/uploads/...
4      https://www.goodcocktails.com/images/drinks/co...
                             ...                        
985    https://cocktailsandshots.com/wp-content/uploa...
986    https://images.absolutdrinks.com/drink-images/...
987    https://cdn.diffords.com/contrib/stock-images/...
988    https://www.haveacocktail.com/images/da/7/7663...
989    https://images.squarespace-cdn.com/content/v1/...
Name: img_url, Length: 990, dtype: object